In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm
import random

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
## read the data:
kaggle=1

if kaggle==1:
    
    train=pd.read_csv("/kaggle/input/cat-in-the-dat/train.csv")
    test=pd.read_csv("/kaggle/input/cat-in-the-dat/test.csv")
    sample=pd.read_csv("/kaggle/input/cat-in-the-dat/sample_submission.csv")
    
else:
    train=pd.read_csv("train.csv")
    test=pd.read_csv("test.csv")
    sample=pd.read_csv("sample_submission.csv")

In [ ]:
train.head()

In [ ]:
train.dtypes

In [ ]:
train.isnull().any(),test.isnull().any()

We dont have any null vales in the dataset.

In [ ]:
for c in train.columns :
    if c not in ['id','target']:
        print(c,"has",train[c].nunique(),"unique values\n")

nom_5,nom_6,nom_7,nom_8,nom_9 has more cardinality compared to other variables.Lets check if there exist any unique values in test set for these columns.

In [ ]:
for col in ['nom_5','nom_6','nom_7','nom_8','nom_9']:
    print(col,"has ",set(train[col].unique())-set(test[col].unique())," unique value in train set not available in test dataset\n")

From the result we understand that nom_7,nom_8 and nom_9 have values which are not present in the test dataset.The local CV score without removing these variable is 0.79910 . 

Now we will start experimenting with different encoding techniques.I will use the techniques described in this [kaggle course](https://www.kaggle.com/matleonard/categorical-encodings)

In [ ]:
train.shape,test.shape

In [ ]:
train.columns,test.columns

## Experiement 2:

Our strategy is to one hot encode the binary categorical variables - bin_0 to bin_4,ordinal variables ord_0 to ord_2 , frequency encode the rest.We remove the cardinality of high cardinality variables by removing the variables present only 0.1 % of the data.

In [ ]:
train.dtypes,test.dtypes

In [ ]:
def reduce_memory(df,col):
    mx = df[col].max()
    print(f'\n reducing memory for {col}')
    if mx<256:
        df[col] = df[col].astype('uint8')
    elif mx<65536:
        df[col] = df[col].astype('uint16')
    else:
        df[col] = df[col].astype('uint32')

In [ ]:



def reduce_cardi(train,test,col):
    print(f'\nBefore reducing the cardinality of col {c} in train was {train[c].nunique()}')
    cv1=pd.DataFrame(train[col].value_counts().reset_index().rename({col:'train'},axis=1))
    cv2=pd.DataFrame(test[col].value_counts().reset_index().rename({col:'test'},axis=1))
    cv3=pd.merge(cv1,cv2,on='index',how='outer')
    factor=len(train)/len(test)
    cv3['train'].fillna(0,inplace=True)
    cv3['test'].fillna(0,inplace=True)
    cv3['remove']=False
    cv3['remove']=cv3['remove']|(cv3['train']<len(train)/10000)  ## remove variables that appear 0.1 % of the total data in train.
    cv3['new'] = cv3.apply(lambda x: x['index'] if x['remove']==False else 0,axis=1)
    cv3['new'],_ = cv3['new'].factorize(sort=True)
    cv3.set_index('index',inplace=True)
    cc = cv3['new'].to_dict()
    train[col] = train[col].map(cc)
    
    test[col]=test[col].map(cc)
    #reduce_memory(test,col)

In [ ]:
card_cols=['nom_9','nom_8','nom_5','nom_6','nom_7']

for c in card_cols:
    print(f'\nReducing cardinality for col {c}')
    reduce_cardi(train,test,c)
    reduce_memory(train,c)
    reduce_memory(test,c)
    print(f'\nAfter reducing the cardinality of col {c} in train is {train[c].nunique()}')

In [ ]:
## Creating new features after doing one round of training and understanding the gini importance

# train['ord_1'+'_'+'ord_5']=train['ord_1']+"_"+train['ord_5']
# test['ord_1'+'_'+'ord_5']=test['ord_1']+"_"+test['ord_5']

# train['ord_1'+'_'+'ord_2']=train['ord_1']+"_"+train['ord_2']
# test['ord_1'+'_'+'ord_2']=test['ord_1']+"_"+test['ord_2']

# train['nom_6'+'_'+'nom_7']=train['nom_6']+'_'+train['nom_7']
# test['nom_6'+'_'+'nom_7']=test['nom_6']+'_'+test['nom_7']

In [ ]:
# a=train
# b=test

In [ ]:
## OHE columns with less cardinality

cat_cols=['bin_0','bin_1','bin_2','bin_3','bin_4','ord_0','ord_1','ord_2']
#https://github.com/rushter/heamy/blob/c330854cee3c547417eb353a4a4a23331b40b4bc/heamy/feature.py

for column in cat_cols:
      
        cate = pd.concat([train[column], test[column]]).dropna().unique()

        train[column] = train[column].astype('category')
        test[column] = test[column].astype('category')

train = pd.get_dummies(train, columns=cat_cols, dummy_na=False, sparse=False)
test = pd.get_dummies(test, columns=cat_cols, dummy_na=False, sparse=False)


In [ ]:
train.columns,test.columns

In [ ]:
import category_encoders as ce

In [ ]:
def frequency_encoding(variable):
    t = train[variable].value_counts().reset_index()
    t = t.reset_index()
    t.loc[t[variable] == 1, 'level_0'] = np.nan
    t.set_index('index', inplace=True)
    max_label = t['level_0'].max() + 1
    t.fillna(max_label, inplace=True)
    return t.to_dict()['level_0']

In [ ]:
# temp = test['nom_6_nom_7'].value_counts().reset_index()
# temp = temp.reset_index()
# temp.loc[temp['nom_6_nom_7'] == 1, 'level_0'] = np.nan
# temp.set_index('index', inplace=True)
# max_label = temp['level_0'].max() + 1
# temp.fillna(max_label, inplace=True)
# #temp.to_dict()['level_0'],max_label
# temp.isna().any()

In [ ]:
ce_cols=['nom_0','nom_1','nom_2','nom_3','nom_4', 'ord_3', 'ord_4', 'ord_5', 'day', 'month']
## Remove day,month and newly created combined columns:
#ce_cols=['nom_0','nom_1','nom_2','nom_3','nom_4','nom_5','nom_6','nom_7','nom_8', 'nom_9', 'ord_3', 'ord_4', 'ord_5']
## Remove cols that have been summarised earlier:
#ce_cols = [c for c in ce_cols if c not in eda_cols]

In [ ]:
ce_cols

In [ ]:
# for c in ce_cols:
#     print(f'\nConverting {c} to lower case')
#     train[c]=train[c].str.lower()
#     test[c]=test[c].str.lower()

In [ ]:
# cat_boost=ce.CatBoostEncoder(cols=ce_cols)
# cat_boost.fit(train[ce_cols],train['target'])

# train=train.join(cat_boost.transform(train[ce_cols]).add_suffix('_cb'))
# test=test.join(cat_boost.transform(test[ce_cols]).add_suffix('_cb'))

In [ ]:
## Frequency encode variables:
for variable in tqdm(ce_cols):
    freq_encod_dict=frequency_encoding(variable)
    train[variable+'_FE']=train[variable].map(lambda x:freq_encod_dict.get(x,1))# return value as 1 if the variable does not exist in freq_encod_dict
    test[variable+'_FE']=test[variable].map(lambda x:freq_encod_dict.get(x,1))
    #categorical_columns.remove(variable)

In [ ]:
train.columns,test.columns

In [ ]:
# test.isnull().any()

In [ ]:
train.head()

In [ ]:
ce_cols

In [ ]:
train=train.drop(columns=ce_cols,axis=1)
test=test.drop(columns=ce_cols,axis=1)

In [ ]:
# train=train.drop(columns=['nom_5','nom_6','nom_7','nom_8','ord_5'],axis=1)
# test=test.drop(columns=['nom_5','nom_6','nom_7','nom_8','ord_5'],axis=1)

In [ ]:
# plt.figure(figsize=(30,30))
# sns.heatmap(train[feats].corr(), cmap='RdBu_r', annot=True, center=0.0)

### Modelling

In [ ]:
##https://www.kaggle.com/kyakovlev/ieee-fe-for-local-test
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [ ]:
seed_everything(1001)

In [ ]:
##import required libraries:
from sklearn.model_selection import StratifiedKFold,GroupKFold
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.linear_model import LogisticRegression
from bayes_opt import BayesianOptimization
import gc
from tqdm import tqdm
#import lofo as lofo

In [ ]:
## Split into train and validation:
y=train['target']
train.drop('target',axis=1,inplace=True)

In [ ]:
n_folds=5
kf=StratifiedKFold(n_splits=n_folds,shuffle=True,random_state=1001)
feats=[f for f in train.columns if f not in ['id','bin_0_1',
 'bin_1_1',
 'bin_2_0',
 'bin_3_F',
 'bin_4_N', 'ord_0_1','ord_1_Novice','ord_2_Lava Hot' ]]  # remove highly correlated features

In [ ]:
oof_preds=np.zeros(train.shape[0])
sub_preds=np.zeros(test.shape[0])

feature_importance_df=pd.DataFrame()
categorical_features=[f for f in train.columns if f not in ['id','bin_0_1',
 'bin_1_1',
 'bin_2_0',
 'bin_3_F',
 'bin_4_N','ord_0_1','ord_1_Novice','ord_2_Lava Hot']]

In [ ]:
categorical_features

In [ ]:
##https://www.kaggle.com/cdeotte/eda-for-columns-v-and-id
plt.figure(figsize=(30,30))
sns.heatmap(train[feats].corr(), cmap='RdBu_r', annot=True, center=0.0)

In [ ]:
cor=train[feats].corr()


In [ ]:
##https://www.kaggle.com/gpreda/santander-eda-and-prediction
c=cor.unstack()
so=c.sort_values(kind='quicksort').reset_index().rename({0:'value'},axis=1)
so=so[so['level_0']!=so['level_1']]
print("Negative correlated features \n",so.head(10),"\nPositive correlated features\n",so.tail(10))

* From the above correlation plot it is seen that the summary features correlate with their respective FE variables.Ord_0_1 , ord_0_2 and ord_0_3 show correlation with each other.

In [ ]:
#Parameters through Bayesian Optimization:

param = {'num_leaves': 40,
         'min_data_in_leaf': 69, 
         'objective':'binary',
         'max_depth': 4,
         'learning_rate': 0.026,
         "boosting": "gbdt",
         "feature_fraction": 0.49,
         "metric": 'auc',
         "lambda_l2": 2.84,
         "random_state": 100,
         "min_gain_to_split":0.386,
         "bagging_freq":5, ## randomly initialized
         "bagging_fraction":0.5,## randomly initialized
         "verbosity": -1}

# {'feature_fraction': 0.49667591307631404,
#  'lambda_l2': 2.843966944905738,
#  'learning_rate': 0.026747986947564972,
#  'max_depth': 4.390096819896572,
#  'min_data_in_leaf': 68.92767604826321,
#  'min_gain_to_split': 0.38669168478099314,
#  'num_leaves': 39.828939074908384}

# param = {'num_leaves': 60,
#          'min_data_in_leaf': 60, 
#          'objective':'binary',
#          'max_depth': -1,
#          'learning_rate': 0.1,
#          "boosting": "gbdt",
#          "feature_fraction": 0.8,
#          "bagging_freq": 1,
#          "bagging_fraction": 0.8 ,
#          "bagging_seed": 11,
#          "metric": 'auc',
#          "lambda_l1": 0.1,
#          "random_state": 133,
#          "verbosity": -1}

In [ ]:
# ## Feature importance:

# dataset=lofo.Dataset(df=train,target="target",features=categorical_features)

# lgbm=lgb.train(param,num_round,
#                     verbose_eval=100,
#                     early_stopping_rounds = 200)

# lf=LOFOImportance(dataset,model=lgbm,cv=4,scoring='roc_auc',n_jobs=4)

# # dataset = Dataset(df=df, target="binary_target", features=features, feature_groups=feature_groups)

# # lgbm = LGBMClassifier(random_state=0, n_jobs=1)

# # lofo = LOFOImportance(dataset, model=lgbm, cv=4, scoring='roc_auc', n_jobs=4)

In [ ]:
## Using the standard cross validation format for training.


for n_folds,(train_idx,valid_idx) in tqdm(enumerate(kf.split(train.values,y.values))):
    print("fold n°{}".format(n_folds+1))
    trn_data = lgb.Dataset(train.iloc[train_idx][feats],
                           label=y.iloc[train_idx],
                           categorical_feature=categorical_features
                          )
    val_data = lgb.Dataset(train.iloc[valid_idx][feats],
                           label=y.iloc[valid_idx],categorical_feature=categorical_features
                          )

    num_round = 10000
    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=100,
                    early_stopping_rounds = 200)
    oof_preds[valid_idx]=clf.predict(train.iloc[valid_idx][feats],num_iteration=clf.best_iteration)
    sub_preds+=clf.predict(test[feats],num_iteration=clf.best_iteration)/kf.n_splits
    
    fold_importance_df=pd.DataFrame()
    fold_importance_df['features']=feats
    fold_importance_df['importance']=clf.feature_importance(importance_type='gain')
    fold_importance_df['folds']=n_folds+1
    print(f'Fold {n_folds+1}: Most important features are:\n')
    for i in np.argsort(fold_importance_df['importance'])[-5:]:
        print(f'{fold_importance_df.iloc[i,0]}-->{fold_importance_df.iloc[i,1]}')
    
    feature_importance_df=pd.concat([feature_importance_df,fold_importance_df],axis=0)
    
    print('Fold %2d AUC : %.6f' % (n_folds + 1, roc_auc_score(y.iloc[valid_idx], oof_preds[valid_idx])))
    del clf
    gc.collect()
    


print('Full auc score %.6f' % (roc_auc_score(y,oof_preds)))

test['target']=sub_preds


* The local cv score increases by 0.000865 when we include only one of the categories in binary cardinality features and leave out using combination of features.The score was .793503 whereas including all the features the score was 0.792638.
* After doing the summary features only for certain columns the local auc score is 0.792114 which is not a improvement over the above which gave a local cv score of 0.793503.Therefore we retain the above score.
* We do not include the summary variables and include only encoded variables in the model and the auc score is 0.791267.
* Removing the highly correlated variables gave auc of 0.788427
* Model score is 0.790089 when we remove the high cardinality variables and do a FE on those variables.

**To try out** 

* Remove either the summary features or the FE features of the correlated variables and see if the score increases or decreases.

* Do a summary features of most important columns https://www.kaggle.com/tunguz/elo-adversarial-validation

* Try removing the variables with high cardinality - https://www.kaggle.com/bogorodvo/lightgbm-baseline-model-using-sparse-matrix

In [ ]:
# ##https://www.kaggle.com/cdeotte/xgb-fraud-with-magic-0-9600
# kf=GroupKFold(n_splits=12)

In [ ]:
# ## Using the standard cross validation format for training.


# for n_folds,(train_idx,valid_idx) in tqdm(enumerate(kf.split(train.values,y.values,groups=train['month']))):
    
#     print("fold n°{}".format(n_folds+1))
#     month = train.iloc[valid_idx]['month'].iloc[0]
#     print('\nFold',n_folds+1,'withholding month',month)
#     print('\n rows of train =',len(train_idx),'rows of holdout =',len(valid_idx))
#     trn_data = lgb.Dataset(train.iloc[train_idx][feats],
#                            label=y.iloc[train_idx],
#                            categorical_feature=categorical_features
#                           )
#     val_data = lgb.Dataset(train.iloc[valid_idx][feats],
#                            label=y.iloc[valid_idx],categorical_feature=categorical_features
#                           )

#     num_round = 10000
#     clf = lgb.train(param,
#                     trn_data,
#                     num_round,
#                     valid_sets = [trn_data, val_data],
#                     verbose_eval=100,
#                     early_stopping_rounds = 200)
#     oof_preds[valid_idx]=clf.predict(train.iloc[valid_idx][feats],num_iteration=clf.best_iteration)
#     sub_preds+=clf.predict(test[feats],num_iteration=clf.best_iteration)/kf.n_splits
    
#     fold_importance_df=pd.DataFrame()
#     fold_importance_df['features']=feats
#     fold_importance_df['importance']=clf.feature_importance(importance_type='gain')
#     fold_importance_df['folds']=n_folds+1
#     print(f'Fold {n_folds+1}: Most important features are:\n')
#     for i in np.argsort(fold_importance_df['importance'])[-5:]:
#         print(f'{fold_importance_df.iloc[i,0]}-->{fold_importance_df.iloc[i,1]}')
    
#     feature_importance_df=pd.concat([feature_importance_df,fold_importance_df],axis=0)
    
#     print('Fold %2d AUC : %.6f' % (n_folds + 1, roc_auc_score(y.iloc[valid_idx], oof_preds[valid_idx])))
#     del clf
#     gc.collect()
    


# print('Full auc score %.6f' % (roc_auc_score(y,oof_preds)))

# test['target']=sub_preds


In [ ]:
# ## Random Forest
# for n_folds,(train_idx,valid_idx) in tqdm(enumerate(kf.split(train.values,y.values))):
#     print("fold n°{}".format(n_folds+1))
#     trn_X = train.iloc[train_idx][feats]
#     trn_Y = y.iloc[train_idx]
#     val_X=train.iloc[valid_idx][feats]
#     val_Y=y.iloc[valid_idx]
#     num_round = 10000
#     clf =RandomForestClassifier(n_estimators=50,min_samples_split=10,min_samples_leaf=10,max_depth=10,random_state=100,criterion='gini',max_features='sqrt',oob_score=True)
#     clf.fit(trn_X,trn_Y)
#     oof_preds[valid_idx]=clf.predict(train.iloc[valid_idx][feats])
#     sub_preds+=clf.predict(test[feats])/kf.n_splits
    
#     fold_importance_df=pd.DataFrame()
#     fold_importance_df['features']=feats
#     fold_importance_df['importance']=clf.feature_importances_
#     fold_importance_df['folds']=n_folds+1
#     print(f'Fold {n_folds+1}: Most important features are:\n')
#     for i in np.argsort(fold_importance_df['importance'])[-5:]:
#         print(f'{fold_importance_df.iloc[i,0]}-->{fold_importance_df.iloc[i,1]}')
    
#     feature_importance_df=pd.concat([feature_importance_df,fold_importance_df],axis=0)
    
#     print('Fold %2d AUC : %.6f' % (n_folds + 1, roc_auc_score(y.iloc[valid_idx], oof_preds[valid_idx])))
#     del clf
#     gc.collect()
    


# print('Full auc score %.6f' % (roc_auc_score(y,oof_preds)))

# test['target']=sub_preds

In [ ]:
sample['target']=sub_preds

In [ ]:
sample.head()

In [ ]:
sample.to_csv("sample_submission.csv",index=False)